In [1]:
# Importing the libraries

In [2]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,LeakyReLU
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# from keras.wrapper.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Activation

In [3]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [4]:
# dummies for geography
geography = pd.get_dummies(dataset[['Geography']],drop_first=True)
gender = pd.get_dummies(dataset[['Gender']],drop_first=True)
# concat dataframes
X = pd.concat([X,geography,gender], axis=1)
# drop geography and gender
X = X.drop(columns=['Geography','Gender'],axis=1)

In [5]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [6]:
# standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(1))
        # model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
#  activation='sigmoid',

In [12]:
# model = KerasClassifier(build_fn=create_model,verbose=0)
model= KerasClassifier(model=create_model, verbose=0, activation='relu', layers=20)

In [13]:
layers =[[20],[40,20],[45,30,15]]
activations = ['sigmoid','relu']
param_grid = dict(layers=layers, activation=activations, batch_size=[128,256],epochs=[30])
grid = GridSearchCV(estimator=model,param_grid=param_grid, cv=5)

In [14]:
grid_result = grid.fit(X_train,y_train)

In [15]:

[grid_result.best_score_,grid_result.best_params_]

[0.804375,
 {'activation': 'relu', 'batch_size': 256, 'epochs': 30, 'layers': [40, 20]}]

In [16]:
pred_y = grid.predict(X_test)

y_pred = (pred_y > 0.5)

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1604,    3],
       [ 374,   19]], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)
score

0.8115